### Author: Russell Ault

# This Notebook Contains code for exploring the functionality of the Pytorch Handwriting Recognition Library

## It is clear to me that I need to write interactive code to load a model that we have trained and test it on a validation set, and have it output accuracy and word error rates. 

## The present way that the library is written is clearly not easily conducive to this

## Here are some comments I have about the code and how to improve it:
- In evaluating model accuracy a character by character accuracy is being used, not an edit distance. I need to put character and word error rates into the model. I should include a mean and sd of these parameters.
- I think I need to just run the validation code right now to see what it does.
- I think that the main python module should be refactored to allow its use in other python modules.

# Reproduce Main Functionality in Notebook fashion

In [49]:
from __future__ import print_function
import argparse
import random
import torch
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import numpy as np
from warpctc_pytorch import CTCLoss
import os
import utils
import dataset

In [50]:
import models.crnn as crnn

SyntaxError: invalid syntax (crnn.py, line 70)

In [ ]:
import sys  
stdout = sys.stdout
reload(sys)  
sys.setdefaultencoding('latin-1')
from model_error import cer, wer


#My workaround was that at the top of the script, I import sys, and store sys.stdout in a separate variable, e.g. stdout.
sys.stdout = stdout
print(sys.getdefaultencoding())

latin-1


In [ ]:
from model_error import cer, wer

In [ ]:
print(3+3)

6


In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--trainroot', required=True, help='path to dataset')
parser.add_argument('--valroot', required=True, help='path to dataset')
parser.add_argument('--workers', type=int, help='number of data loading workers', default=2)
parser.add_argument('--batchSize', type=int, default=64, help='input batch size')
parser.add_argument('--imgH', type=int, default=32, help='the height of the input image to network')
parser.add_argument('--imgW', type=int, default=100, help='the width of the input image to network')
parser.add_argument('--nh', type=int, default=256, help='size of the lstm hidden state')
parser.add_argument('--niter', type=int, default=25, help='number of epochs to train for')
parser.add_argument('--lr', type=float, default=0.01, help='learning rate for Critic, default=0.00005')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for adam. default=0.5')
parser.add_argument('--cuda', action='store_true', help='enables cuda')
parser.add_argument('--ngpu', type=int, default=1, help='number of GPUs to use')
parser.add_argument('--crnn', default='', help="path to crnn (to continue training)")
parser.add_argument('--alphabet', type=str, default='0123456789abcdefghijklmnopqrstuvwxyz')
parser.add_argument('--experiment', default=None, help='Where to store samples and models')
parser.add_argument('--displayInterval', type=int, default=500, help='Interval to be displayed')
parser.add_argument('--n_test_disp', type=int, default=10, help='Number of samples to display when test')
parser.add_argument('--valInterval', type=int, default=500, help='Interval to be displayed')
parser.add_argument('--saveInterval', type=int, default=500, help='Interval to be displayed')
parser.add_argument('--adam', action='store_true', help='Whether to use adam (default is rmsprop)')
parser.add_argument('--adadelta', action='store_true', help='Whether to use adadelta (default is rmsprop)')
parser.add_argument('--keep_ratio', action='store_true', help='whether to keep ratio for image resize')
parser.add_argument('--random_sample', action='store_true', help='whether to sample the dataset with random sampler')
opt = parser.parse_args()
print(opt)

SystemExit: 2

In [ ]:
if experiment is None:
    experiment = 'expr'
os.system('mkdir {0}'.format(experiment))

### Set variables

In [ ]:
trainroot = "/home/ubuntu/russell/nephi/data/lmdb/train"
valroot = "/home/ubuntu/russell/nephi/data/lmdb/val"
batchSize = 64
nh = 256                  # size of the LSTM hidden state
imgW = 100
imgH = 32
ngpu = 1
beta1 = 0.5
lr = 0.0001
workers = 10
keep_ratio = True
adam = True
adadelta = False
n_test_disp = 100

alph_file_dylan = "/home/ubuntu/dylan/nephi/alphabet.txt"
alph_file_russell = "/home/ubuntu/russell/nephi/alphabet.txt"
alphabet = '0123456789abcdefghijklmnopqrstuvwxyzB- EÂ¬Ã¼.RSÅ«J/DHA:K¤¿ZLGFNTPCOVWIM<8d>Ä<81><9f>,<93>È³¶'
#0123456789abcdefghijklmnopqrstuvwxyzW VCGū¬.HM,ILAZ:BTÿSER<BC>JFāP<9F>NDKOȳ<B6>
#<A4><8D>()—̈-<84><93>Q<96>/Y<BE>U<>+  # This is what I got from Dylan's file

untrained_crnn_dylan = "/home/ubuntu/dylan/nephi/expr/netCRNN_1_100.pth"
#less trained 29
lesstrained_crnn_dylan = "/home/ubuntu/dylan/nephi/expr/netCRNN_1000_100.pth"
trained_crnn_russell = "/home/ubuntu/russell/nephi/expr/netCRNN_3870_100.pth"
trained_crnn_dylan = "/home/ubuntu/dylan/nephi/expr/netCRNN_3210_100.pth"

In [ ]:
manualSeed = random.randint(1, 10000)  # fix seed
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
np.random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [ ]:
cudnn.benchmark = True
cuda = True

#if torch.cuda.is_available() and not cuda:
#    print("WARNING: You have a CUDA device, so you should probably run with --cuda")

train_dataset = dataset.lmdbDataset(root=trainroot)
sampler = dataset.randomSequentialSampler(train_dataset, batchSize)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batchSize, sampler=sampler,
    num_workers=int(workers),
    collate_fn=dataset.alignCollate(imgH=imgH, imgW=imgW, keep_ratio=keep_ratio))

training_eval_set = dataset.lmdbDataset(
    root=trainroot, transform=dataset.resizeNormalize((imgW, imgH))) 
test_dataset = dataset.lmdbDataset(
    root=valroot, transform=dataset.resizeNormalize((imgW, imgH)))   # I have changed this line from the original code

In [ ]:
from PIL import Image
import torchvision.transforms as transforms
class Normalize(object):

    def __init__(self, size, interpolation=Image.BILINEAR):
        self.size = size
        self.interpolation = interpolation
        self.toTensor = transforms.ToTensor()

    def __call__(self, img):
        #img = img.resize(self.size, self.interpolation)
        img = self.toTensor(img)
        img.sub_(0.5).div_(0.5)
        return img

In [ ]:
train_test_loader = dataset.lmdbDataset(root=trainroot, transform=Normalize((imgW, imgH)))

trainer = torch.utils.data.DataLoader(
    train_test_loader, batch_size=batchSize, sampler=sampler,
    num_workers=int(workers),)


In [ ]:
train_test_loader
trainer = iter(train_test_loader)

In [ ]:
data = trainer.next()
cpu_images, _ = data

In [ ]:
cpu_images.numpy()

array([[[ 0.87450981,  0.87450981,  0.83529413, ...,  1.        ,
          1.        ,  1.        ],
        [ 0.81176472,  0.81176472,  0.82745099, ...,  1.        ,
          1.        ,  1.        ],
        [ 0.8509804 ,  0.85882354,  0.89019608, ...,  1.        ,
          1.        ,  1.        ],
        ..., 
        [ 1.        ,  1.        ,  1.        , ...,  0.89019608,
          0.88235295,  0.89019608],
        [ 1.        ,  1.        ,  1.        , ...,  0.9137255 ,
          0.87450981,  0.84313726],
        [ 1.        ,  1.        ,  1.        , ...,  0.92156863,
          0.85882354,  0.80392158]]], dtype=float32)

### Questions on image resizing:
- Does the keep ratio resizing do the largest aspect ratio within a batch, or within the whole dataset (I think batch)?
- For applying same approach to test set, do i need to use the largest training aspect ratio, or, if training aspect ratio is by batch, do I just do largest by batch within the test set?
- Can the CNN-RNN take any dimension? (different widths for different batches, or even every photo being a different width)?
- With the rules of matrix multiplication, it makes sense to me that the aspect ratio would have to be the same in a given batch, else you couldn't use a matrix for the RNN part (and maybe the CNN too, depending on how it is calculated)
      
### From this morning's thoughts (23 February 2018):
- I think the best first stab at a complete solution would be to use the BYU heigh size and just let the code do renormalizing on its own.
- also, I should have the validation function load a data loader, rather than an lmdb dataset, or I just make sure the same aspect ratio thing is done.
- Well, with the training set, I just want to load the original training batches, given how the aspect ratio has been done, validation can do their own thing

In [ ]:
#trainer
#max_iter = min(max_iter, len(data_loader))
#max_iter = len(data_loader)
image_count = 0
widths = []
heights = []
for i in range(len(train_test_loader)):
    data = trainer.next()
    i += 1
    cpu_images, cpu_texts = data
    batch_size = cpu_images.size(0)
    image_count = image_count + batch_size
    _, h, w = cpu_images
    widths.append(w)
    heights.append(h)
    
    #if self.keep_ratio:
     #       ratios = []
    # ##       for image in images:
    #            w, h = image.size
    #            ratios.append(w / float(h))
    #        ratios.sort()
    #        max_ratio = ratios[-1]
    #        imgW = int(np.floor(max_ratio * imgH))
            
            #RA: I don't understand the purpose of this line, and for handwriting recognition imgW >= imgH
            #imgW = max(imgH * self.min_ratio, imgW)  # assure imgH >= imgW


ValueError: need more than 1 value to unpack

In [ ]:
image_count

In [ ]:
# Load in the two alphabets
alphabet_russell = ''
alphabet_dylan = ''

with open(alph_file_russell, 'r') as myfile:
    alphabet_russell = myfile.read()
with open(alph_file_dylan, 'r') as myfile:
    alphabet_dylan = myfile.read()
    


In [ ]:
# Test the dylan ALphabet and model first
alphabet = alphabet_dylan

nclass = len(alphabet) + 1
nc = 1

converter = utils.strLabelConverter(alphabet)
criterion = CTCLoss()

In [ ]:
# custom weights initialization called on crnn
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [ ]:
crnn = crnn.CRNN(imgH, nc, nclass, nh)

In [ ]:
crnn.apply(weights_init)



# Based on the above unexpected key error, I will assume that when I try to run the original code with a validation epoch number, I will get the same error

In [ ]:
image = torch.FloatTensor(batchSize, 3, imgH, imgH)
text = torch.IntTensor(batchSize * 5)          # RA: I don't understand why the text has this size
length = torch.IntTensor(batchSize)

if cuda:
    crnn.cuda()
    crnn = torch.nn.DataParallel(crnn, device_ids=range(ngpu))
    image = image.cuda()
    criterion = criterion.cuda()

In [ ]:
# Load Dylan's pretrained model first
trained_crnn = trained_crnn_dylan
if trained_crnn != '':
    print('loading pretrained model from %s' % trained_crnn)
    crnn.load_state_dict(torch.load(trained_crnn))
print(crnn)

In [ ]:
image = Variable(image)
text = Variable(text)
length = Variable(length)

# loss averager
loss_avg = utils.averager()

# setup optimizer
if adam:
    optimizer = optim.Adam(crnn.parameters(), lr=lr,
                           betas=(beta1, 0.999))
elif adadelta:
    optimizer = optim.Adadelta(crnn.parameters(), lr=lr)
else:
    optimizer = optim.RMSprop(crnn.parameters(), lr=lr)

# Here is where I will test out the code.

### First order of business is to see what val outputs currently on these pretrained models using the test set.
### Then add word and character error rate and a way to calculate mean and standard deviation of them.

In [ ]:
def val(net, dataset, criterion, max_iter=100):
    print('Start val')

    for p in crnn.parameters():
        p.requires_grad = False

    net.eval()
    data_loader = torch.utils.data.DataLoader(
        dataset, shuffle=True, batch_size=batchSize, num_workers=int(workers))
    val_iter = iter(data_loader)

    i = 0
    n_correct = 0
    loss_avg = utils.averager()
    
    image_count = 0
    
    # Character and word error rate lists
    char_error = []
    w_error = []

    max_iter = min(max_iter, len(data_loader))
    #max_iter = len(data_loader)
    for i in range(max_iter):
        data = val_iter.next()
        i += 1
        cpu_images, cpu_texts = data
        batch_size = cpu_images.size(0)
        image_count = image_count + batch_size
        utils.loadData(image, cpu_images)
        t, l = converter.encode(cpu_texts)
        utils.loadData(text, t)
        utils.loadData(length, l)

        preds = crnn(image)
        preds_size = Variable(torch.IntTensor([preds.size(0)] * batch_size))
        cost = criterion(preds, text, preds_size, length) / batch_size
        loss_avg.add(cost)
        
        
        # RA: While I am not sure yet, it looks like a greedy decoder and not beam search is being used here
        # Also, a simple character by character accuracy is being used, not an edit distance.
        # Case is ignored in the accuracy, which is not ideal for an actual working system
        
        _, preds = preds.max(2)
        preds = preds.squeeze(2)
        preds = preds.transpose(1, 0).contiguous().view(-1)
        sim_preds = converter.decode(preds.data, preds_size.data, raw=False)
        for pred, target in zip(sim_preds, cpu_texts):
            if pred == target.lower():
                n_correct += 1
            #print(pred)
            #print("Pred: %s; target: %s" % (pred, target))
            char_error.append(cer(pred, target.lower()))
            w_error.append(wer(pred, target.lower()))

    raw_preds = converter.decode(preds.data, preds_size.data, raw=True)[:n_test_disp]
    for raw_pred, pred, gt in zip(raw_preds, sim_preds, cpu_texts):
        print('%-20s => %-20s, gt: %-20s' % (raw_pred, pred, gt))

    accuracy = n_correct / float(max_iter * batchSize)
    print('Test loss: %f, accuracy: %f' % (loss_avg.val(), accuracy))
    
    char_arr =np.array(char_error)
    w_arr = np.array(w_error)
    #numpy.std(arr, ddof=1)
    #numpy.mean(arr, axis=0)
    #print("All character error rates:")
    #print(char_error)
    #print("All word error rates")
    #print(w_error)
    print("Character error rate mean: %4.4f; Character error rate sd: %4.4f" % (np.mean(char_arr), np.std(char_arr, ddof=1)))
    print("Word error rate mean: %4.4f; Word error rate sd: %4.4f" % (np.mean(w_arr), np.std(w_arr, ddof=1)))
    print("Total number of images in validation set: %8d" % image_count)
    return (char_error, w_error)

In [ ]:
def trainBatch(net, criterion, optimizer):
    data = train_iter.next()
    cpu_images, cpu_texts = data
    batch_size = cpu_images.size(0)
    utils.loadData(image, cpu_images)
    t, l = converter.encode(cpu_texts)
    utils.loadData(text, t)
    utils.loadData(length, l)

    preds = crnn(image)
    preds_size = Variable(torch.IntTensor([preds.size(0)] * batch_size))
    cost = criterion(preds, text, preds_size, length) / batch_size
    crnn.zero_grad()
    cost.backward()
    optimizer.step()
    return cost

### Now see how the pre-trained model works on the validation set
oops, I have to figure out how to change the kernal of this python notebook...

In [ ]:
val(crnn, test_dataset, criterion)

In [ ]:
val(crnn_dylan_trained, test_dataset, criterion)

In [ ]:
# Now with the character and word error rates
char_e, w_e = val(crnn, test_dataset, criterion)

In [ ]:
crnn.load_state_dict(torch.load(lesstrained_crnn_dylan))

In [ ]:
char_e, w_e = val(crnn, test_dataset, criterion)

# Next things to do:
1) Make the word error and character error code robust to empty sets ("such as give a dummy variable if length < 1")
2) Incorporate character and word error rates into the training set too

In [ ]:
trained_crnn = trained_crnn_dylan
if trained_crnn != '':
    print('loading pretrained model from %s' % trained_crnn)
    crnn.load_state_dict(torch.load(trained_crnn))


In [ ]:
char_e, w_e = val(crnn, test_dataset, criterion)

In [ ]:
char_e
char_a = np.array(char_e)


In [ ]:
print(char_a)
print("%4.3f" % np.mean(char_a))

In [ ]:
#crnn_dylan_trained = crnn
crnn_dylan_trained

### As of 17 February 2018, the machine is learning to read. It is rough, but it is learning. This is exciting!

In [ ]:
for epoch in range(niter):
    train_iter = iter(train_loader)
    i = 0
    while i < len(train_loader):
        for p in crnn.parameters():
            p.requires_grad = True
        crnn.train()

        cost = trainBatch(crnn, criterion, optimizer)
        loss_avg.add(cost)
        i += 1

        if i % displayInterval == 0:
            print('[%d/%d][%d/%d] Loss: %f' %
                  (epoch, niter, i, len(train_loader), loss_avg.val()))
            loss_avg.reset()

        if i % valInterval == 0:
            val(crnn, test_dataset, criterion)

        # do checkpointing
        if i % saveInterval == 0:
            torch.save(
                crnn.state_dict(), '{0}/netCRNN_{1}_{2}.pth'.format(experiment, epoch, i))

# Next Goal: Loop over all trained dylan models and see word error rate and character error rate on training and test data
### I will have a dictionary for training and a dictionary for test results, each of which will have a dictionary for character error mean and SD and word error mean and SD

In [ ]:
! ls expr

In [ ]:
import glob
#glob.glob('expr/*')

# We are focusing on dylan's models here
model_path = '/home/ubuntu/dylan/nephi/expr/'
train_results = {}
test_results = {}
for model in glob.glob(model_path + '*'):
    # Load the weights of the model
    print('loading pretrained model from %s' % (model_path + model))
    crnn.load_state_dict(torch.load(model))     # In my first run through of this code, this was incorrectly trained_crnn. Now it is correct.
    
    char_error, w_error = val(crnn, test_dataset, criterion)
    test_results[model] = {"char" : (np.mean(char_error), np.std(char_error, ddof=1)),
                          "word": (np.mean(w_error), np.std(w_error, ddof=1))}
    char_error, w_error = val(crnn, training_eval_set, criterion)
    train_results[model] = {"char" : (np.mean(char_error), np.std(char_error, ddof=1)),
                          "word": (np.mean(w_error), np.std(w_error, ddof=1))}
    print("Finished model: %s" % model)
    
# Now let's write the output to a csv file
with open ('dylan_results.csv', 'w') as f:
    f.write('Model, Dataset, Character Error (mean), Character Error (sd), Word Error (mean), Word Error (sd)\n')
    for key, value in test_results.items():
        f.write("%s, %s, %4.4f, %4.4f, %4.4f, %4.4f\n" % (key, 'validation', value['char'][0], value['char'][1], value['word'][0], value['word'][1]))
        
    for key, value in train_results.items():
        f.write("%s, %s, %4.4f, %4.4f, %4.4f, %4.4f\n" % (key, 'training', value['char'][0], value['char'][1], value['word'][0], value['word'][1]))

# I think putting all this in a panda spreadsheet would probably be best. I'll do this later

In [ ]:
nums = (1.4, 2.5, 3.5)

print("%f, %f, %f, %f" % (1.3, nums[1], nums[2], nums[0] ))

# The Current Goal for 20 Feb 2018 is to be able to test a model on all images and sort images by character and/or word error rates, displaying images, predictions, ground truth and character and word error rates

## I am going to try to reverse engineer the validation function to get at the images at this point. I think this is the fastest way

In [ ]:
net = crnn
dataset = test_dataset

In [ ]:
print('Start val')

for p in crnn.parameters():
    p.requires_grad = False

net.eval()
data_loader = torch.utils.data.DataLoader(
    dataset, shuffle=True, batch_size=batchSize, num_workers=int(workers))
val_iter = iter(data_loader)

In [ ]:
i = 0
n_correct = 0
loss_avg = utils.averager()

image_count = 0

# Character and word error rate lists
char_error = []
w_error = []
max_iter = 100
max_iter = min(max_iter, len(data_loader))
#max_iter = len(data_loader)
#for i in range(max_iter):

In [ ]:
data = val_iter.next()
i += 1
cpu_images, cpu_texts = data
batch_size = cpu_images.size(0)
image_count = image_count + batch_size

In [ ]:
print(image)
print(cpu_images)

### I am following the instructions at these websites to display a pytorch tensor as an image
https://discuss.pytorch.org/t/how-to-visualize-display-a-data-image-in-torch-floattensor-type/7770

https://matplotlib.org/users/image_tutorial.html#plotting-numpy-arrays-as-images


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

In [ ]:
images_np = cpu_images.numpy()

image_1 = images_np[0, :, :, :]
print(image_1.shape)

In [ ]:
image_reshape = np.swapaxes(image_1, 0, 2)
image_reshape = np.swapaxes(image_reshape, 0, 1)
print(image_reshape.shape)
image_reshape = np.squeeze(image_reshape)
print(image_reshape.shape)

In [ ]:
image_reshape.shape

In [ ]:
from skimage.color import gray2rgb, rgb2gray
#print(gray2rgb(image_reshape).shape)
#imgplot = plt.imshow(image_reshape)
plt.imshow(image_reshape, cmap='gray')

In [ ]:
skimage.color.gray2rgb(image, alpha=None)

###### So, I can print a grey-scale image of any tensor in this project. So I will load all images in a validation run in a list, together with predictions and ground truth as well as word and error rates. Then I can find word and character rates that I want to display, and i can graph accordingly.

The following files at fastai can help me with graphing things how I want to:

https://github.com/fastai/courses/blob/master/deeplearning1/nbs/utils.py

https://github.com/fastai/courses/blob/master/deeplearning2/utils2.py


In [ ]:
def to_grayscale(img):
    image_reshape = np.swapaxes(img, 0, 2)
    image_reshape = np.swapaxes(image_reshape, 0, 1)
    image_reshape = np.squeeze(image_reshape)
    return(image_reshape)

def val(net, dataset, criterion, max_iter=100):
    print('Start val')

    for p in crnn.parameters():
        p.requires_grad = False

    net.eval()
    data_loader = torch.utils.data.DataLoader(
        dataset, shuffle=True, batch_size=batchSize, num_workers=int(workers))
    val_iter = iter(data_loader)

    i = 0
    n_correct = 0
    loss_avg = utils.averager()
    
    image_count = 0
    
    
    
    # Character and word error rate lists
    char_error = []
    w_error = []
    
    # Lists of images, predictions and ground truth to correlate with character and word error rates
    image_list = []
    pred_list = []
    gt_list = []
    
    

    max_iter = min(max_iter, len(data_loader))
    #max_iter = len(data_loader)
    for i in range(max_iter):
        data = val_iter.next()
        i += 1
        cpu_images, cpu_texts = data
        batch_size = cpu_images.size(0)
        image_count = image_count + batch_size
        utils.loadData(image, cpu_images)
        t, l = converter.encode(cpu_texts)
        utils.loadData(text, t)
        utils.loadData(length, l)

        preds = crnn(image)
        preds_size = Variable(torch.IntTensor([preds.size(0)] * batch_size))
        cost = criterion(preds, text, preds_size, length) / batch_size
        loss_avg.add(cost)
        
        
        # RA: While I am not sure yet, it looks like a greedy decoder and not beam search is being used here
        # Also, a simple character by character accuracy is being used, not an edit distance.
        # Case is ignored in the accuracy, which is not ideal for an actual working system
        
        _, preds = preds.max(2)
        preds = preds.squeeze(2)
        preds = preds.transpose(1, 0).contiguous().view(-1)
        sim_preds = converter.decode(preds.data, preds_size.data, raw=False)
        for pred, target, img in zip(sim_preds, cpu_texts, cpu_images.numpy()):
            if pred == target.lower():
                n_correct += 1
            #print(pred)
            #print("Pred: %s; target: %s" % (pred, target))
            char_error.append(cer(pred, target.lower()))
            w_error.append(wer(pred, target.lower()))
            image_list.append(to_grayscale(img))
            pred_list.append(pred)
            gt_list.append(target)

    raw_preds = converter.decode(preds.data, preds_size.data, raw=True)[:n_test_disp]
    for raw_pred, pred, gt in zip(raw_preds, sim_preds, cpu_texts):
        print('%-20s => %-20s, gt: %-20s' % (raw_pred, pred, gt))

    accuracy = n_correct / float(max_iter * batchSize)
    print('Test loss: %f, accuracy: %f' % (loss_avg.val(), accuracy))
    
    char_arr =np.array(char_error)
    w_arr = np.array(w_error)
    #numpy.std(arr, ddof=1)
    #numpy.mean(arr, axis=0)
    #print("All character error rates:")
    #print(char_error)
    #print("All word error rates")
    #print(w_error)
    print("Character error rate mean: %4.4f; Character error rate sd: %4.4f" % (np.mean(char_arr), np.std(char_arr, ddof=1)))
    print("Word error rate mean: %4.4f; Word error rate sd: %4.4f" % (np.mean(w_arr), np.std(w_arr, ddof=1)))
    print("Total number of images in validation set: %8d" % image_count)
    return (char_error, w_error, image_list, pred_list, gt_list)

In [ ]:
char_error, w_error, all_images, all_preds, all_gts = val(crnn, training_eval_set, criterion)

NameError: name 'crnn' is not defined

In [ ]:
#https://gist.github.com/soply/f3eec2e79c165e39c9d540e916142ae1
def show_images(images, cols = 1, titles = None):
    """Display a list of images in a single figure with matplotlib.
    
    Parameters
    ---------
    images: List of np.arrays compatible with plt.imshow.
    
    cols (Default = 1): Number of columns in figure (number of rows is 
                        set to np.ceil(n_images/float(cols))).
    
    titles: List of titles corresponding to each image. Must have
            the same length as titles.
    """
    assert((titles is None)or (len(images) == len(titles)))
    n_images = len(images)
    if titles is None: titles = ['Image (%d)' % i for i in range(1,n_images + 1)]
    fig = plt.figure()
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(cols, np.ceil(n_images/float(cols)), n + 1)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    plt.show()

In [ ]:
1023/2

In [ ]:
def plots_extreme(char_err, w_err, images, preds, gts, n=5, err="char", best=True, median=False):
    # Fixing random state for reproducibility
    np.random.seed(19680801)

    # Ascending sort
    to_sort = None
    if err=="char":
        to_sort = char_err
    elif err == "word":
        to_sort = w_err
    elif err == "both":
        to_sort = [j/2 for j in (char_err + w_err)]
    s_idxs = [i[0] for i in sorted(enumerate(to_sort), key=lambda x:x[1], reverse = False if best else True)]
    s_char_err = [char_err[i] for i in s_idxs]
    s_w_err = [w_err[i] for i in s_idxs]
    s_images = [images[i] for i in s_idxs]
    s_preds = [preds[i] for i in s_idxs]
    s_gts = [gts[i] for i in s_idxs]
    
    titles = ["Prediction: %-20s\nGround Truth: %-20s" % (pred, gt) for pred, gt in zip(s_preds, s_gts)]
    #for raw_pred, pred, gt in zip(raw_preds, sim_preds, cpu_texts):
        #print('%-20s => %-20s, gt: %-20s' % (raw_pred, pred, gt))

    print("Got through all the sorting in plots_best")
    if median:
        show_images(s_images[len(s_images)/2:len(s_images)/2+n], cols=5, titles=titles[len(s_images)/2:len(s_images)/2+n])
    else:
        show_images(s_images[0:n], cols=5, titles=titles[0:n])   
    return(1)
    
    
    # Need to give show images all correct order

In [ ]:
sorts = np.argsort(char_error)

sortes = [i[0] for i in sorted(enumerate(char_error), key=lambda x:x[1], reverse=True)]
#for idx in sorts:
#    print(char_error[sorts])

In [ ]:
for i in sortes:
    print(char_error[i])
    

In [ ]:
[j/2 for j in (w_error + char_error)]

In [ ]:
plt.hist(w_error, range=(0, 2), bins=10)

In [ ]:
char_error[0:5]

In [ ]:
plots_extreme(char_error, w_error, all_images, all_preds, all_gts, n=5, err="char", best=False)  # maybe a both char and word too using a linear combination
#plots_average()   # plots around the average I really just need to sort indices
#plots_worst

In [ ]:
char_error, w_error, all_images, all_preds, all_gts = val(crnn, test_dataset, criterion)

In [ ]:
plots_extreme(char_error, w_error, all_images, all_preds, all_gts, n=5, err="char", best=True, median=True) 

In [ ]:
utils.loadData(image, cpu_images)
t, l = converter.encode(cpu_texts)
utils.loadData(text, t)
utils.loadData(length, l)

In [ ]:
preds = crnn(image)
preds_size = Variable(torch.IntTensor([preds.size(0)] * batch_size))
cost = criterion(preds, text, preds_size, length) / batch_size
loss_avg.add(cost)


# RA: While I am not sure yet, it looks like a greedy decoder and not beam search is being used here
# Also, a simple character by character accuracy is being used, not an edit distance.
# Case is ignored in the accuracy, which is not ideal for an actual working system

_, preds = preds.max(2)
preds = preds.squeeze(2)
preds = preds.transpose(1, 0).contiguous().view(-1)
sim_preds = converter.decode(preds.data, preds_size.data, raw=False)
for pred, target in zip(sim_preds, cpu_texts):
    if pred == target.lower():
        n_correct += 1
    #print(pred)
    #print("Pred: %s; target: %s" % (pred, target))
    char_error.append(cer(pred, target.lower()))
    w_error.append(wer(pred, target.lower()))

raw_preds = converter.decode(preds.data, preds_size.data, raw=True)[:n_test_disp]
for raw_pred, pred, gt in zip(raw_preds, sim_preds, cpu_texts):
print('%-20s => %-20s, gt: %-20s' % (raw_pred, pred, gt))

accuracy = n_correct / float(max_iter * batchSize)
print('Test loss: %f, accuracy: %f' % (loss_avg.val(), accuracy))

char_arr =np.array(char_error)
w_arr = np.array(w_error)
#numpy.std(arr, ddof=1)
#numpy.mean(arr, axis=0)
#print("All character error rates:")
#print(char_error)
#print("All word error rates")
#print(w_error)
print("Character error rate mean: %4.4f; Character error rate sd: %4.4f" % (np.mean(char_arr), np.std(char_arr, ddof=1)))
print("Word error rate mean: %4.4f; Word error rate sd: %4.4f" % (np.mean(w_arr), np.std(w_arr, ddof=1)))
print("Total number of images in validation set: %8d" % image_count)
return (char_error, w_error)

# For Oliver:
#### The data and model are loaded. Here's the code to generate all the pictures of the powerpoint. The call to val() will take a minute fore the training set

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

In [ ]:
# Training set pictures
char_error, w_error, all_images, all_preds, all_gts = val(crnn, training_eval_set, criterion)

# Plot 5 of the best pictures by character error rate
plots_extreme(char_error, w_error, all_images, all_preds, all_gts, n=5, err="char", best=True, median=False) 

# Plot 5 of the worst pictures
plots_extreme(char_error, w_error, all_images, all_preds, all_gts, n=5, err="char", best=False, median=False)

# Plot 5 pictures around the median performance
plots_extreme(char_error, w_error, all_images, all_preds, all_gts, n=5, err="char", best=True, median=True) 

In [ ]:
# Validation set pictures
char_error, w_error, all_images, all_preds, all_gts = val(crnn, test_dataset, criterion)

# Plot 5 of the best pictures by character error rate
plots_extreme(char_error, w_error, all_images, all_preds, all_gts, n=5, err="char", best=True, median=False) 

# Plot 5 of the worst pictures
plots_extreme(char_error, w_error, all_images, all_preds, all_gts, n=5, err="char", best=False, median=False)

# Plot 5 pictures around the median performance
plots_extreme(char_error, w_error, all_images, all_preds, all_gts, n=5, err="char", best=True, median=True) 

In [ ]:
char_error[1]